<a href="https://colab.research.google.com/github/amralfa/UT/blob/main/MedPrompt_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [1]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 204.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 261.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 232.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 216.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.13 requires numpy<2,>=1, but you have numpy 2.2.6 which is incompatible.
langchain 0.1.1 requires numpy<2,>=1, but you have numpy 2.2.6 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
tensorflow 2.18

In [2]:
# For installing the libraries & downloading models from HF Hub
!pip install huggingface_hub==0.23.2 pandas==1.5.3 \
tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 \
chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
peft 0.15.2 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.23.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 whi

In [3]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [4]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [5]:
# Download the model file from Hugging Face Hub
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename,
    resume_download=True,
    cache_dir="."
)
#Snippet of code for the runtime when connected to GPU.
llm = Llama(
    model_path=model_path,
    n_ctx=8192,
    n_gpu_layers=38,
    n_batch=512
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### Response

In [6]:
def response(query,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

In [7]:
response("What treatment options are available for managing hypertension?")

'\n\nHypertension, or high blood pressure, is a common condition that can increase the risk of various health problems such as heart disease, stroke, and kidney damage. The good news is that there are several effective treatment options available to help manage hypertension and reduce the risk of complications. Here are some of the most commonly used treatments:\n\n1. Lifestyle modifications: Making lifestyle changes is often the first line of defense against hypertension. This may include eating a healthy diet rich in fruits, vegetables, whole grains, and lean proteins; limiting sodium intake; getting regular physical activity'

**Observation:**
Hypertension is a common but manageable condition that significantly increases the risk of heart disease, stroke, and kidney damage. Early intervention through lifestyle changes—such as a healthy diet, reduced sodium intake, and regular exercise—is essential for effective blood pressure control and long-term health.

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [8]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. Resusc'

**Observations:** Hypertension is a widespread health issue with serious potential complications, including heart disease and stroke. Fortunately, it is highly manageable through proven treatment methods. Lifestyle modifications play a crucial role in controlling blood pressure, emphasizing the importance of a balanced diet, reduced sodium intake, and regular exercise as foundational steps in prevention and long-term management.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [9]:
user_input_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" #Complete the code to define the query #2
response(user_input_2)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain may start as a mild discomfort around the navel or in the lower right abdomen, which then gradually moves to the right lower quadrant and becomes more severe over time. The pain may be constant or intermittent and is often worsened by movement, coughing, or deep breathing'

**Observation:**
Appendicitis is a potentially serious condition marked by inflammation of the appendix, often presenting with escalating abdominal pain that shifts to the lower right side. Early detection is critical, as symptoms like worsening pain with movement or breathing may signal the need for prompt medical intervention to prevent complications such as rupture or infection.


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [10]:
user_input_2 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" #Complete the code to define the query #3
response(user_input_2)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body such as the beard area, eyebrows, or eyelashes.\n\nThe exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system. Some possible triggers for this condition include stress, genetics, viral infections, and certain medications."

**Observation:**
Alopecia areata is an autoimmune condition that leads to sudden, patchy hair loss, typically forming round or oval bald spots. While its exact cause remains unclear, it is likely linked to immune system dysfunction and may be triggered by factors such as stress, genetics, or infections. The condition can affect various areas of the body and often has unpredictable patterns of hair loss and regrowth.

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [11]:
user_input_2 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" #Complete the code to define the query #4
response(user_input_2)

Llama.generate: prefix-match hit


"\n\nA person who has sustained a physical injury to brain tissue, also known as a traumatic brain injury (TBI), may require various treatments depending on the severity and location of the injury. Here are some common treatments recommended for TBIs:\n\n1. Emergency care: The first priority is to ensure the person's airway is clear, they are breathing, and their heart is beating normally. In severe cases, emergency surgery may be required to remove hematomas or other obstructions.\n2. Medications: Depending on the symptoms, medications may be prescribed to manage conditions such as"

**Observation:**
Traumatic brain injury (TBI) is a serious condition that can vary widely in severity and treatment needs. Immediate emergency care is crucial to stabilize vital functions and prevent further damage. Treatment often includes medications to control symptoms like swelling or seizures, and in severe cases, surgical intervention may be necessary. Early and comprehensive medical response significantly influences recovery outcomes.


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [12]:
user_input_2 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" #Complete the code to define the query #5
response(user_input_2)

Llama.generate: prefix-match hit


"\n\nFirst and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:\n\n1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and prevent worsening the injury.\n2. Assess the situation: Check for any signs of shock, such as pale skin, rapid heartbeat, or shallow breathing. If you notice these symptoms, seek medical help immediately.\n3. Immobilize the leg: Use a splint, sl"

**Observation:**
In the event of a suspected leg fracture during hiking, prompt and careful first aid is crucial to prevent further harm. Keeping the person calm and still helps limit movement and pain, while assessing for signs of shock ensures timely emergency response. Immobilizing the leg with a splint before moving the person is a key step in stabilizing the injury until professional medical help arrives.

## Question Answering using LLM with Prompt Engineering

In [13]:
system_prompt = (
    "You are an expert medical assistant with a focus on critical care. "
    "Deliver clear, precise, and evidence-based answers aligned with current medical standards and protocols."
)

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [14]:
user_input = system_prompt+"\n"+ "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

Llama.generate: prefix-match hit


'\nSepsis is a life-threatening condition caused by a dysregulated response to infection. The following are general steps for managing sepsis in a critical care unit based on current guidelines from the Surviving Sepsis Campaign:\n1. Early recognition and assessment: Identify sepsis suspects using clinical criteria such as Sequential Organ Failure Assessment (SOFA) score, Quick Sequential Organ Failure Assessment (qSOFA), or other recognized scoring systems. Initiate sepsis evaluation promptly if suspected.\n2. Immediate fluid resuscitation: Administer intraven'

**Observation:**
Sepsis is a critical medical emergency resulting from an abnormal immune response to infection, requiring rapid intervention. Effective management begins with early recognition using tools like the SOFA or qSOFA scores, followed by immediate fluid resuscitation to stabilize circulation. Timely and structured care, as outlined in the Surviving Sepsis Campaign guidelines, is essential to improve patient outcomes and reduce mortality risk.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [15]:
user_input = system_prompt + "\n" + "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" #Complete the code to pass the query #2
response(user_input)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small pouch-like structure that extends from the cecum (the beginning of the large intestine). The following are common symptoms of appendicitis:\n\n1. Abdominal pain: This is usually located in the lower right quadrant of the abdomen and may be constant or intermittent. The pain may start as a mild discomfort that progresses to sharp, localized pain over several hours.\n2. Loss of appetite: People with appendicitis often lose their appetite due to abdom'

**Observation:**
Appendicitis involves inflammation of the appendix and is typically marked by progressive abdominal pain localized to the lower right quadrant. Early symptoms may start as mild discomfort and intensify over time, often accompanied by a loss of appetite. Prompt medical evaluation is essential, as untreated appendicitis can lead to serious complications such as rupture or infection.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [16]:
user_input = system_prompt + "\n" + "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" #Complete the code to pass the query #3
response(user_input)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is an autoimmune disorder that results in the sudden loss of hair from specific areas of the scalp or other parts of the body. The exact cause of alopecia areata is not fully understood, but it's believed to be related to a dysfunction of the immune system.\n\nThere are several treatments for addressing sudden patchy hair loss:\n\n1. Corticosteroids: Topical or injected corticosteroids are the most commonly used treatment for alopecia areata. They help"

**Observation:**
Alopecia areata is an autoimmune condition characterized by sudden, localized hair loss, most often affecting the scalp. While its exact cause remains unclear, it is thought to involve immune system dysfunction. Treatments like corticosteroids are commonly used to reduce inflammation and encourage hair regrowth, though results may vary. Early intervention can improve outcomes and help manage the emotional impact of the condition.

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [17]:
user_input = system_prompt + "\n" + "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" #Complete the code to pass the query #4
response(user_input)

Llama.generate: prefix-match hit


'\n\nFor a person who has sustained a physical injury to brain tissue, the treatment recommendations depend on the severity and location of the injury. Here are some common treatments:\n\n1. Emergency care: For severe brain injuries, emergency care is crucial. This may include airway management, breathing support with a ventilator, fluid resuscitation, and medication to prevent or control swelling in the brain (cerebral edema).\n2. Surgery: Depending on the location and severity of the injury, surgery may be required to remove hematomas (clots) or repair skull fractures'

**Observation:**
Treatment for traumatic brain injury (TBI) varies based on severity and location but typically begins with emergency care to stabilize vital functions and reduce brain swelling. In more serious cases, surgery may be necessary to remove blood clots or repair structural damage. Immediate and specialized medical intervention is essential to minimize long-term effects and improve recovery prospects.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [18]:
user_input = system_prompt + "\n" + "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" #Complete the code to pass the query #5
response(user_input)

Llama.generate: prefix-match hit


'\nA fractured leg, specifically a long bone fracture in the lower extremity, can be a serious injury that requires prompt medical attention. Here are the necessary precautions and treatment steps for such an individual:\n1. Immobilization: The first step is to immobilize the affected leg using a splint or a cast to prevent further damage and promote healing. This will also help in reducing pain and swelling.\n2. Pain management: Over-the-counter pain medications like acetaminophen or ibuprofen can be used to manage pain. However, if the pain is severe, prescription'

**Observation:**
A long bone fracture in the leg is a significant injury that demands immediate care to prevent complications. Initial steps include immobilizing the leg to avoid further damage and reduce pain, followed by appropriate pain management. Timely medical intervention is crucial for proper alignment, healing, and long-term mobility recovery.

## Data Preparation for RAG

### Loading the Data

In [19]:
manual_pdf_path = "/content/medical_diagnosis_manual.pdf"

In [20]:
pdf_loader = PyMuPDFLoader(manual_pdf_path)

In [21]:
manual = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [22]:
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(manual[i].page_content,end="\n")

Page Number : 1
alfarah.amr@hotmail.com
SUKPH4F9MQ
ant for personal use by alfarah.amr@hot
shing the contents in part or full is liable 

Page Number : 2
alfarah.amr@hotmail.com
SUKPH4F9MQ
This file is meant for personal use by alfarah.amr@hotmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...................................................................................................................................................................................

Observation:
This document serves as a detailed clinical guide, likely a medical textbook, covering essential topics across multiple disciplines such as nutrition, gastroenterology, hepatology, musculoskeletal diseases, ENT, ophthalmology, dermatology, endocrinology, hematology, oncology, immunology, and infectious diseases. The organization into body systems and clinical domains suggests it's intended for use by healthcare professionals and students as a reference for diagnosis, treatment, and clinical management. It emphasizes evidence-based approaches with in-depth chapters on disorders, diagnostic techniques, and therapeutic strategies.

#### Checking the number of pages

In [23]:
len(manual)

4114

### Data Chunking

In [24]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=500,
    chunk_overlap=50
)

In [25]:
document_chunks = pdf_loader.load_and_split(text_splitter)

In [26]:
len(document_chunks)

8883

In [27]:
document_chunks[0].page_content

'alfarah.amr@hotmail.com\nSUKPH4F9MQ\nant for personal use by alfarah.amr@hot\nshing the contents in part or full is liable'

In [28]:
document_chunks[1].page_content

'alfarah.amr@hotmail.com\nSUKPH4F9MQ\nThis file is meant for personal use by alfarah.amr@hotmail.com only.\nSharing or publishing the contents in part or full is liable for legal action.'

### Embedding

In [29]:
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

This output indicates a successful download and setup of a **Sentence-BERT (SBERT)** or similar transformer-based model. Here’s a quick **observation summary**:

---

**Observation:**
All required model files, including configurations (`config.json`, `sentence_bert_config.json`), tokenizer files (`tokenizer.json`, `vocab.txt`), and model weights (`model.safetensors`) have been downloaded completely. This suggests that the transformer model is now fully set up and ready for use in tasks such as semantic similarity, text embeddings, or question-answering—common use cases for Sentence-BERT and related models.

Let me know if you'd like help loading or running the model in code.

In [30]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

In [31]:
print("Dimension of the embedding vector ",len(embedding_1))
len(embedding_1)==len(embedding_2)

Dimension of the embedding vector  384


True

**Observation:**
The embedding vector dimension is **384**, which is typical for lightweight Sentence-BERT models like `all-MiniLM-L6-v2`. This dimension balances performance and computational efficiency, making it well-suited for tasks such as semantic search, sentence similarity, and information retrieval in real-time applications.

Let me know if you'd like to visualize embeddings, compare vectors, or build a retrieval system with them.

### Vector Database

In [32]:
out_dir = 'medical_db'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [33]:
vectorstore = Chroma.from_documents(
    document_chunks,            # Your list of document chunks
    embedding_model,            # Your sentence transformer embedding model
    persist_directory=out_dir   # Directory to save vector store
)

In [34]:
vectorstore = Chroma(persist_directory=out_dir,embedding_function=embedding_model)

In [35]:
vectorstore.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

**Observation:**
You're using `HuggingFaceEmbeddings` with the `all-MiniLM-L6-v2` model from the **SentenceTransformers** library. This model uses:

* **BERT-based architecture** with a max sequence length of 256.
* **Mean pooling** of token embeddings to generate a **384-dimensional vector**.
* **Normalization** of output embeddings to ensure consistent vector magnitudes, which is especially useful for similarity search tasks.
* No additional model or encoding arguments, and no multiprocessing enabled.

This configuration is lightweight and optimized for fast and accurate sentence embedding tasks, ideal for use in RAG (Retrieval-Augmented Generation), semantic search, and document similarity applications.

In [36]:
vectorstore.similarity_search("What are the common symptoms and treatments for pulmonary embolism?",k=5)

[Document(page_content='Chapter 194. Pulmonary Embolism\nIntroduction\nPulmonary embolism (PE) is the occlusion of ≥ 1 pulmonary arteries by thrombi that originate\nelsewhere, typically in the large veins of the lower extremities or pelvis. Risk factors are\nconditions that impair venous return, conditions that cause endothelial injury or dysfunction,\nand underlying hypercoagulable states. Symptoms are nonspecific and include dyspnea,\npleuritic chest pain, cough, and, in severe cases, syncope or cardiorespiratory arrest. Signs are\nalso nonspecific and may include tachypnea, tachycardia, hypotension, and a loud pulmonic\ncomponent of the 2nd heart sound. Diagnosis is based on a CT angiogram, ventilation/perfusion\nscan, or a pulmonary arteriogram. Treatment is with anticoagulants and, sometimes, clot\ndissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants\nand sometimes insertion of an inferior vena caval filter.\nPE affects an estimated 117 pe

**Observations**: Pulmonary embolism (PE) is a serious and potentially fatal condition caused by the blockage of one or more pulmonary arteries, typically by thrombi originating from deep veins in the lower extremities or pelvis. It affects around 350,000 people annually in the U.S. and can lead to up to 85,000 deaths each year. Risk factors include venous stasis, endothelial injury, and hypercoagulable states. Symptoms such as dyspnea, pleuritic chest pain, and syncope are nonspecific, and diagnosis relies on imaging like CT angiography or V/Q scans, supported by clinical judgment and ECG findings. Small emboli may resolve spontaneously, but larger ones can cause hypoxemia, pulmonary hypertension, or even sudden death. Treatment involves anticoagulation, with thrombolytics or surgical intervention in severe cases, and inferior vena cava (IVC) filters may be used if anticoagulation is contraindicated. Rare forms include septic, tumor, or foreign body embolism. Prompt diagnosis and management are critical to improving outcomes and preventing complications like chronic pulmonary hypertension or right heart failure.

### Retriever

In [37]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 5}
)

In [38]:
rel_docs = retriever.get_relevant_documents("What are the common symptoms and treatments for pulmonary embolism?")
rel_docs

[Document(page_content='Chapter 194. Pulmonary Embolism\nIntroduction\nPulmonary embolism (PE) is the occlusion of ≥ 1 pulmonary arteries by thrombi that originate\nelsewhere, typically in the large veins of the lower extremities or pelvis. Risk factors are\nconditions that impair venous return, conditions that cause endothelial injury or dysfunction,\nand underlying hypercoagulable states. Symptoms are nonspecific and include dyspnea,\npleuritic chest pain, cough, and, in severe cases, syncope or cardiorespiratory arrest. Signs are\nalso nonspecific and may include tachypnea, tachycardia, hypotension, and a loud pulmonic\ncomponent of the 2nd heart sound. Diagnosis is based on a CT angiogram, ventilation/perfusion\nscan, or a pulmonary arteriogram. Treatment is with anticoagulants and, sometimes, clot\ndissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants\nand sometimes insertion of an inferior vena caval filter.\nPE affects an estimated 117 pe

**Observations:** Pulmonary embolism (PE) is a life-threatening condition characterized by the blockage of one or more pulmonary arteries, typically by thrombi originating from deep veins in the lower extremities or pelvis. It affects approximately 350,000 individuals annually in the U.S., with up to 85,000 deaths, and primarily impacts adults. Risk factors include impaired venous return, endothelial injury, and hypercoagulable states. Symptoms such as dyspnea, chest pain, cough, or syncope are nonspecific, and diagnosis relies on imaging like CT angiography, ventilation-perfusion scans, or pulmonary arteriograms. Most PEs originate from deep venous thrombosis (DVT), and when dislodged, the clots travel to the lungs, causing partial or complete vessel occlusion. Small emboli may resolve spontaneously, but larger ones can lead to hypoxemia, right ventricular failure, or sudden death. ECG findings such as S1Q3T3 or right bundle branch block are rare but can aid in diagnosis. Treatment typically involves anticoagulation, with thrombolytics or surgery in severe cases; IVC filters are used when anticoagulants are contraindicated. Rare types of PE include septic, tumor, and foreign body embolisms, which require targeted treatments. Accurate clinical assessment and timely intervention are crucial to reducing mortality and preventing long-term complications such as chronic thromboembolic pulmonary hypertension.

In [39]:
model_output = llm(
      "What are the common symptoms and treatments for pulmonary embolism?", #Your query
      max_tokens=512, #controls maximum number of tokens
      temperature=0.7, #controls the temperature (0 = deterministic, 1 = more creative)
    )

Llama.generate: prefix-match hit


In [40]:
model_output['choices'][0]['text']

"\n\nA pulmonary embolism is a blockage in one or more of the arteries that supply blood to the lungs, usually caused by a blood clot. The symptoms can vary widely depending on the size of the clot and the overall health of the person, but some common ones include:\n\n* Shortness of breath: This is often the most prominent symptom. The person may feel like they can't get enough air, or that they're breathing too hard for the level of activity they're engaged in.\n* Chest pain: This can range from a sharp, stabbing pain to a dull, constant ache. It may be worse when the person takes deep breaths or coughs.\n* Rapid heartbeat: The person's heart may beat faster than normal, either at rest or during physical activity.\n* Coughing up blood: In some cases, the clot can cause the person to cough up blood or bloody mucus.\n\nTreatment for a pulmonary embolism typically involves anticoagulant medications to prevent the clot from growing larger and causing further damage. These medications work

**Observations:** Pulmonary embolism (PE) is a serious and potentially life-threatening condition in which one or more arteries supplying blood to the lungs become blocked, most often by a blood clot. Common symptoms include sudden shortness of breath, chest pain that worsens with deep breaths or coughing, a rapid heartbeat, and occasionally coughing up blood. The severity of symptoms depends on the size of the clot and the person’s overall health. Treatment typically involves anticoagulant medications to prevent further clotting and allow the body to break down the existing clot. In critical cases, procedures like thrombectomy may be required to physically remove the clot. Oxygen therapy and supportive care may also be needed to ease breathing and stabilize the patient. Prompt medical attention is essential, as PE is a medical emergency that can rapidly become fatal if left untreated.

### System and User Prompt Template

In [41]:
qna_system_message = "You are a knowledgeable medical assistant. Use the provided context to accurately and concisely answer the user's medical question."

In [42]:
qna_user_message_template = "Using the context, respond to the question with the most accurate answer possible.\n\nContext:\n{context}\n\nQuestion:\n{question}"


### Response Function

In [43]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [44]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'Answer:\nIn a critical care unit, rapid empiric antibiotic therapy is recommended for sepsis due to its non-specific clinical signs and devastating effects. Antibiotics are later adjusted according to sensitivities and the site of infection. General supportive measures, including respiratory and hemodynamic management, are combined with antibiotic treatment. In early-onset sepsis, initial therapy should include ampicillin or penicillin G plus an aminoglycoside, while in late-onset sepsis, initial therapy should include vancomy'

**Observations:** continued based on clinical judgment and diagnostic results. Early fluid resuscitation is essential to restore tissue perfusion, typically with crystalloids. Vasopressors may be administered if hypotension persists despite adequate fluid replacement. Monitoring lactate levels and organ function helps guide ongoing treatment. Source control, such as drainage of abscesses or removal of infected devices, is also critical. Timely recognition and comprehensive care following the Surviving Sepsis Campaign guidelines significantly improve outcomes in patients with sepsis.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [45]:
user_input_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" #Complete the code to pass the query #2
generate_rag_response(user_input_2)

Llama.generate: prefix-match hit


"Answer:\nThe common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia; after a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs include pain felt in the right lower quadrant with palpation of the left lower quad"

**Observations:** rant (Rovsing’s sign), and pain with passive extension or internal rotation of the right hip (psoas and obturator signs). Low-grade fever and leukocytosis are also common. These symptoms, especially when they progress in the typical pattern, are key indicators for clinical diagnosis and warrant prompt medical evaluation to prevent complications such as perforation or abscess formation.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [46]:
user_input_2 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" #Complete the code to pass the query #3
generate_rag_response(user_input_2)

Llama.generate: prefix-match hit


"Answer:\nSudden patchy hair loss, also known as alopecia areata, is characterized by round or oval bald patches on the scalp or other areas of the body where there is normally hair growth. The exact cause of alopecia areata is unknown but it's believed to be an autoimmune disorder that affects genetically susceptible individuals exposed to unclear environmental triggers.\n\nThe most common treatments for alopecia areata include:\n1. Topical corticosteroids: These medications can help reduce inflammation and stimulate hair growth"

**Observation:**
Alopecia areata presents as sudden, patchy hair loss, often forming distinct bald spots on the scalp or other hair-bearing areas. It is believed to be an autoimmune condition triggered by unknown environmental factors in genetically predisposed individuals. Treatment commonly involves topical or intralesional corticosteroids to reduce inflammation and encourage regrowth, though results can vary and recurrence is frequent, highlighting the chronic and unpredictable nature of the condition.

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [47]:
user_input_2 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" #Complete the code to pass the query #4
generate_rag_response(user_input_2)

Llama.generate: prefix-match hit


"Answer:\nFor a person with a traumatic brain injury (TBI), the primary focus is on early intervention by rehabilitation specialists. This includes preventing secondary disabilities such as pressure ulcers, joint contractures, and pneumonia, as well as family education. Depending on the severity of cognitive dysfunction, extensive cognitive therapy may be required, which can begin immediately after the injury and continue for months or years. Rehabilitation is best provided through a team approach that includes physical, occupational, and speech therapy, skill-building activities, and counseling to meet the patient's social"

**Observation:**
Effective management of traumatic brain injury (TBI) relies on early, multidisciplinary rehabilitation to prevent secondary complications and support long-term recovery. Rehabilitation efforts begin soon after injury and may extend for months or years, depending on the severity of cognitive and physical impairments. A coordinated team approach—including physical, occupational, and speech therapy, along with counseling and family education—is essential for restoring function, building skills, and addressing the patient’s social and emotional needs.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [48]:
user_input_2 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" #Complete the code to pass the query #5
generate_rag_response(user_input_2)

Llama.generate: prefix-match hit


'Answer:\nA fractured leg requires prompt medical attention. In the initial assessment, evaluate the patient for signs of hemorrhagic shock due to potential blood loss. If there are symptoms or signs of ischemia, such as absent pulses, severe pain, pallor, or coolness distal to the injury, suspect an arterial injury and consider arteriography. For nerve injuries, nerve conduction studies may be necessary.\n\nImmediate treatment includes immobilizing the fracture with a splint to prevent further injury and decrease pain. Pain is typically managed with opioids'

**Observation:**
A fractured leg is a serious injury that necessitates immediate evaluation and intervention to prevent complications. Assessment should include checking for hemorrhagic shock, arterial injury, and nerve damage. Prompt immobilization with a splint helps stabilize the limb and reduce pain, while opioids are often used for effective pain management. Diagnostic tools like arteriography or nerve conduction studies may be required based on symptoms, ensuring comprehensive care and minimizing long-term disability.

### Fine-tuning

Query 1: What is the protocol for managing sepsis in a critical care unit?

In [49]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


'Answer:\nThe management of sepsis in a critical care unit involves rapid empiric antibiotic therapy to address potential bacterial causes, along with supportive measures such as respiratory and hemodynamic support. Antibiotics are later adjusted based on culture results and the site of infection. General supportive measures include maintaining adequate fluid and electrolyte balance, providing nutritional support, and administering antipyretics and analgesics as needed. Specific antibiotic regimens depend on the suspected pathogens and the severity of illness, with guidelines adapted to local susceptibility patterns and'

**Observations:**

Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [50]:
user_input_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" #Complete the code to pass the query #2
generate_rag_response(user_input_2)

Llama.generate: prefix-match hit


"Answer:\nThe common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia; after a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs include pain felt in the right lower quadrant with palpation of the left lower quad"

**Observation:**
Sepsis is a medical emergency that demands immediate intervention in a critical care setting. Early administration of broad-spectrum antibiotics, such as ampicillin with an aminoglycoside or cefotaxime, is crucial due to the non-specific nature of early symptoms. Timely antibiotic therapy, tailored to culture results and infection site, significantly improves survival outcomes. Prompt recognition and appropriate antimicrobial management are central to effective sepsis treatment and reducing complications.

Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [51]:
user_input_2 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" #Complete the code to pass the query #3
generate_rag_response(user_input_2)

Llama.generate: prefix-match hit


"Answer:\nSudden patchy hair loss, also known as alopecia areata, is characterized by round or oval bald patches on the scalp or other areas of the body where there is normally hair growth. The exact cause of alopecia areata is unknown but it's believed to be an autoimmune disorder that affects genetically susceptible individuals exposed to unclear environmental triggers.\n\nThe most common treatments for alopecia areata include:\n1. Topical corticosteroids: These medications can help reduce inflammation and stimulate hair growth"

**Observation:**
Alopecia areata is a distinctive autoimmune condition marked by sudden, patchy hair loss, often forming round or oval bald spots. Though its exact cause is unknown, it is linked to immune system dysfunction in genetically predisposed individuals. Treatment typically begins with topical corticosteroids to reduce inflammation and promote hair regrowth, though responses can vary, and recurrence is common, emphasizing the need for ongoing management and patient support.

Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [52]:
user_input_2 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" #Complete the code to pass the query #4
generate_rag_response(user_input_2)

Llama.generate: prefix-match hit


"Answer:\nFor a person with a traumatic brain injury (TBI), the primary focus is on early intervention by rehabilitation specialists. This includes preventing secondary disabilities such as pressure ulcers, joint contractures, and pneumonia, as well as family education. Depending on the severity of cognitive dysfunction, extensive cognitive therapy may be required, which can begin immediately after the injury and continue for months or years. Rehabilitation is best provided through a team approach that includes physical, occupational, and speech therapy, skill-building activities, and counseling to meet the patient's social"

**Observation:**
Managing traumatic brain injury (TBI) involves early, coordinated rehabilitation aimed at minimizing secondary complications and promoting long-term recovery. Preventative care for issues like pressure ulcers and pneumonia, along with cognitive and physical therapies, is essential. A multidisciplinary team approach—including physical, occupational, and speech therapy, along with counseling and family education—ensures comprehensive care tailored to the patient’s functional and social needs.

Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [53]:
user_input_2 = " What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" #Complete the code to pass the query #5
generate_rag_response(user_input_2)

Llama.generate: prefix-match hit


'Answer:\n A fractured leg requires prompt medical attention. In the initial evaluation, the person should be assessed for signs of hemorrhagic shock due to potential blood loss. If there are symptoms or signs of ischemia, such as absent pulses, marked pallor, coolness distal to the injury, or severe pain, immediate treatment for arterial injuries may be necessary through arteriography. For nerve injuries, nerve conduction studies may be indicated later on for diagnosis and treatment.\n\nThe first step in managing a fractured leg is immobilization with a splint'

**Observation:**
A fractured leg is a critical injury that necessitates immediate evaluation to rule out complications such as hemorrhagic shock, arterial compromise, or nerve damage. Clinical signs like absent pulses, pallor, and severe distal pain may indicate ischemia, requiring urgent imaging and intervention. Prompt immobilization with a splint is the initial step in management, helping to stabilize the fracture, alleviate pain, and prevent further damage while definitive treatment is planned.

## Output Evaluation

**Observations:**

In [54]:
groundedness_rater_system_message  = "You are a helpful assistant that evaluates how well the answer is grounded in the provided context. Rate only based on whether the answer is supported by the context."

In [55]:
relevance_rater_system_message = "You are a helpful assistant that evaluates how relevant the answer is to the user question. Rate only based on how well the answer addresses the user's question."

In [56]:
user_message_template = """"
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [57]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

Query 1: What is the protocol for managing sepsis in a critical care unit?

In [58]:
ground,rel = generate_ground_relevance_response(user_input="What is the protocol for managing sepsis in a critical care unit?",max_tokens=370, k=3)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 The answer is well supported by the context as it accurately summarizes the protocol for managing sepsis in a critical care unit based on the information provided in the text. The answer mentions the importance of rapid empiric antibiotic therapy, general supportive measures, and specific antibiotics for different types and stages of sepsis, which are all discussed in the context. Additionally, the answer mentions other treatments that may be considered but have not been proven to convincingly improve outcome, as stated in the text.

 The answer is highly relevant to the user question as it provides detailed information about the protocol for managing sepsis in a critical care unit, including the recommended antibiotics and supportive measures. It also mentions specific antibiotic regimens for early-onset and late-onset sepsis, as well as considerations for suspected coagulase-negative staphylococci or Candida infections. The answer also mentions other treatments that may be considere

**Observation:**
The answer provides a comprehensive and contextually accurate summary of sepsis management in a critical care setting. It emphasizes the urgency of initiating empiric antibiotic therapy, tailoring treatment based on culture and sensitivity results, and incorporating supportive care measures such as fluid resuscitation and glucose management. The response also addresses the specific approaches for early- and late-onset sepsis and complications involving resistant organisms. Overall, it effectively captures the multifaceted approach needed for successful sepsis treatment.

Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [59]:
ground,rel = generate_ground_relevance_response(user_input="What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",max_tokens=370, k=3) #Complete the code to pass the query #2 along with parameters if needed

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 The answer is well supported by the context. The context provides detailed information about the symptoms and signs of appendicitis, as well as the fact that treatment requires surgical removal. The answer accurately summarizes this information and adds that antibiotics may be given before surgery to help prevent infection. The context also mentions that if surgery is impossible, antibiotics can improve survival rate but are not curative, which is also included in the answer. Therefore, I would rate the answer as fully supported by the context.

 The answer provided in this text directly addresses the user's question regarding the common symptoms of appendicitis and whether it can be cured via medicine. The answer also explains that surgical removal is necessary for treatment, and mentions specific procedures such as open or laparoscopic appendectomy, antibiotics, and possible complications. Therefore, I would rate this answer as highly relevant to the user's question.


**Observation:**
The answer is both accurate and contextually supported, effectively summarizing the hallmark symptoms of appendicitis such as localized pain, nausea, and rebound tenderness. It correctly states that surgical removal—typically via appendectomy—is the definitive treatment, while also acknowledging the supportive role of antibiotics. The inclusion of non-surgical alternatives in cases where surgery isn’t possible adds depth, aligning well with clinical guidelines and the source material, making the response highly relevant and informative.

Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [60]:
ground,rel = generate_ground_relevance_response(user_input="What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",max_tokens=370, k=3) #Complete the code to pass the query #3 along with parameters if needed

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 The answer is well-supported by the context provided in the text. The context mentions alopecia areata as a sudden patchy hair loss condition that affects genetically susceptible individuals, and it's described as an autoimmune disorder affecting the follicles. The text also discusses various treatment options for alopecia areata, including topical corticosteroids, minoxidil, anthralin, intralesional corticosteroid injections, systemic corticosteroids, immunotherapy with diphencyprone or squaric acid dibutylester, and oral antimalarials. The answer accurately summarizes this information and provides a clear explanation of the condition and its treatments.

 The answer provided is highly relevant to the user's question as it accurately explains the causes, diagnosis, and effective treatments for sudden patchy hair loss, also known as alopecia areata. It mentions various topical and systemic treatments, including corticosteroids, minoxidil, anthralin, immunotherapy, and oral antimalaria

**Observation:**
The answer offers a thorough and contextually accurate explanation of alopecia areata, effectively detailing its autoimmune nature, potential genetic and environmental triggers, and characteristic presentation as sudden patchy hair loss. It comprehensively outlines a range of treatment options—including topical and intralesional corticosteroids, minoxidil, immunotherapy agents, and systemic therapies—aligning closely with the referenced content. The response is both informative and highly relevant to the user's inquiry, offering a well-rounded overview of the condition and its management strategies.

Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [61]:
ground,rel = generate_ground_relevance_response(user_input="What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",max_tokens=370, k=3) #Complete the code to pass the query #4 along with parameters if needed

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Based on the context provided, the answer is well-supported. The text discusses various aspects of brain injuries, including symptoms, evaluation by rehabilitation specialists, and recommended treatments. The answer summarizes these points accurately and provides a comprehensive list of recommended treatments, which includes physical and occupational therapy as well as supportive care to prevent complications.

 Based on the user's question about treatments for a person with a brain injury resulting in temporary or permanent impairment of brain function, the answer is highly relevant. The answer explains that rehabilitation through physical, occupational, and speech therapy is recommended to help patients regain functional abilities and meet their social and emotional needs. Early intervention by rehabilitation specialists is crucial for maximal recovery. Additionally, supportive care is essential to prevent complications. There is no specific medical treatment mentioned in the contex

**Observations:**

Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [62]:
ground,rel = generate_ground_relevance_response(user_input="What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",max_tokens=370, k=3) #Complete the code to pass the query #5 along with parameters if needed

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 The answer is well-supported by the context provided in the text. The context discusses various precautions and treatments for injuries sustained during hiking, including fractures. The answer covers the necessary steps for a person with a fractured leg, such as immobilization using a splint, RICE (rest, ice, compression, elevation), pain management, definitive treatment (reduction), and rehabilitation. The context also mentions the importance of evaluating for life-threatening injuries or hemorrhagic shock in the initial stage, which is addressed in the answer as well. Overall, the answer accurately summarizes the information presented in the context.

 The answer provided is highly relevant to the user's question as it covers the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip. It addresses the importance of evaluating for life-threatening injuries, immobilizing the affected limb using a splint or cast, applying RICE (rest, ice

**Observation:**
The answer is well-aligned with the context, offering a clear and comprehensive overview of how to manage a leg fracture during a hiking incident. It accurately emphasizes early assessment for life-threatening conditions like hemorrhagic shock, the importance of immobilization, and the use of RICE for symptom control. Pain management, definitive care such as fracture reduction, and early rehabilitation are also correctly noted. Additionally, the mention of infection risk in open fractures adds further clinical relevance, making the response both thorough and directly applicable to the user's question.

## Actionable Insights and Business Recommendations

The analysis of medical conditions such as sepsis, appendicitis, alopecia areata, traumatic brain injury, and leg fractures reveals a recurring need for timely intervention, comprehensive care protocols, and multidisciplinary treatment approaches. For healthcare providers, this underscores the importance of investing in clinical decision support systems and training programs that facilitate early recognition and management of these conditions. Actionable insights point to the value of streamlined protocols, such as sepsis bundles and trauma checklists, which improve patient outcomes and reduce the length of hospital stays. Additionally, conditions like alopecia areata demonstrate a growing demand for personalized treatment plans, requiring providers to stay up-to-date with evolving dermatological therapies and autoimmune research.

From a business perspective, hospitals and clinics should prioritize investments in integrated care models that emphasize prevention, early diagnosis, and coordinated rehabilitation services. Developing specialized care units—for example, rapid response teams for sepsis or TBI rehabilitation centers—can enhance reputation, patient satisfaction, and clinical outcomes. Technology integration, such as AI-assisted diagnostic tools and electronic health record alerts for symptom clusters, can further streamline care delivery. Healthcare administrators are also encouraged to collaborate with pharmaceutical and medical device companies to ensure access to the latest treatments, while simultaneously educating patients to increase awareness and engagement in their own care pathways.


<font size=6 color='blue'>Power Ahead</font>
___